In [ ]:
import os
from pathlib import Path
import torch
import re
import random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam

In [9]:
MAX_LEN = 64

## Loading all data into memory

corpus_movie_conv = '.\datasets\movie_conversations.txt'
corpus_movie_lines = '.\datasets\movie_lines.txt'



# Splitting text using special lines
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

lines_dict = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dict[objects[0]] = objects[-1]

# generating the question answer pair
with open (corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
    
pairs=[]
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        if i == len(ids) -1:
            break
        first = lines_dict[ids[i]].strip()
        second = lines_dict[ids[i+1]].strip()
        
        qa_pairs.append(''.join(first.strip()[:MAX_LEN]))
        qa_pairs.append(''.join(second.strip()[:MAX_LEN]))
        pairs.append(qa_pairs)

In [21]:
# WordPiece tokenizer

# save data as txt file
text_data = []
file_count = 0

for sample in tqdm.tqdm([x[0] for x in pairs]):
    text_data.append(sample)
    
    # once we hit 10K mark, save to file
    if len(text_data) == 10_000:
        with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
        
paths = [str(x) for x in Path('./data').glob('**/*.txt')]


100%|██████████| 221616/221616 [00:00<00:00, 1066740.81it/s]


In [23]:
# train own tokenizer
tokenizer= BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)
    
tokenizer.train(
    files=paths,
    vocab_size=30_000,
    min_frequency=5,
    limit_alphabet=1000,
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]','[UNK]']
)    

tokenizer.save_model('./bert-it-1','bert-it')
tokenizer = BertTokenizer.from_pretrained('./bert-it-1/bert-it-vocab.txt', local_files_only=True)


c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:2165: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
